# TODO:

* Move the code from 'new_models.py' to calendar_event and schedule_draft files.
* move the 
* make new helper functions for:
    - receive json generated by the timebox agent and convert to a ScheduleDraft
    - save the ScheduleDraft to the database
* make a new agent that receives the draft id and queries the database for that draft and then
  saves it to the calendar.

# Testing with a real schedule

# storing that draft

In [5]:
from typing import List
import json
from fateforger.agents.schedular.models import CalendarEvent
from fateforger.agents.schedular.models.schedule_draft import ScheduleDraft, DraftStore

events = [CalendarEvent.model_validate(event) for event in json.loads(timebox_json)]

event =  events[-1]
event.model_dump(exclude_none=True)


# gives str

{'calendarId': 'primary',
 'summary': 'Final prep',
 'start': datetime.datetime(2025, 7, 30, 22, 55),
 'end': datetime.datetime(2025, 7, 30, 23, 0),
 'timeZone': 'Europe/Amsterdam',
 'colorId': '3'}

# Committing that draft to google calendar

# New definitions

In [9]:
import pprint
from typing import List
from pydantic import TypeAdapter

schema: Dict[str, Any] = TypeAdapter(List[CalendarEvent]).json_schema(
    schema_generator=LLMJsonSchema,
    mode="validation"
)
pprint.pprint(schema)

{'$defs': {'Attendee': {'additionalProperties': False,
                        'properties': {'email': {'description': 'Email address '
                                                                'of the '
                                                                'attendee',
                                                 'format': 'email',
                                                 'title': 'Email',
                                                 'type': 'string'}},
                        'required': ['email'],
                        'title': 'Attendee',
                        'type': 'object'},
           'CalendarEvent': {'properties': {'attendees': {'anyOf': [{'items': {'$ref': '#/$defs/Attendee'},
                                                                     'type': 'array'},
                                                                    {'type': 'null'}],
                                                          'default': None,
                  

In [1]:
import json
timebox_json = '''
[
  {
    "event_type": "H",
    "summary": "Sleep",
    "start": "2025-08-02T03:00:00+02:00",
    "duration": "PT8H"
  },
  {
    "event_type": "H",
    "summary": "Morning routine",
    "duration": "PT1H"
  },
  {
    "event_type": "DW",
    "summary": "Deep Work – Partitional Dissonance",
    "duration": "PT2H"
  },
  {
    "event_type": "H",
    "summary": "Meditation",
    "duration": "PT10M"
  },
  {
    "event_type": "H",
    "summary": "Gym session",
    "duration": "PT1H45M"
  },
  {
    "event_type": "PR",
    "summary": "Personal planning session",
    "duration": "PT20M"
  },
  {
    "event_type": "R",
    "summary": "Lunch & buffer",
    "duration": "PT30M"
  },
  {
    "event_type": "SW",
    "summary": "Market trip",
    "duration": "PT1H30M"
  },
  {
    "event_type": "PR",
    "summary": "Planning session with girlfriend",
    "duration": "PT20M"
  },
  {
    "event_type": "BU",
    "summary": "Buffer before dinner",
    "duration": "PT25M"
  },
  {
    "event_type": "SW",
    "summary": "Cook dinner",
    "duration": "PT1H"
  },
  {
    "event_type": "BU",
    "summary": "Buffer before Baldur's Gate",
    "duration": "PT30M"
  },
  {
    "event_type": "R",
    "summary": "Play Baldur's Gate with girlfriend",
    "start": "2025-08-02T20:30:00+02:00",
    "duration": "PT1H30M"
  },
  {
    "event_type": "H",
    "summary": "Evening hygiene",
    "duration": "PT30M"
  },
  {
    "event_type": "SW",
    "summary": "Pack for next day",
    "duration": "PT30M"
  },
  {
    "event_type": "R",
    "summary": "Sci-fi reading",
    "duration": "PT1H",
    "end": "2025-08-03T00:00:00+02:00"
  },
  {
    "event_type": "H",
    "summary": "Sleep",
    "start": "2025-08-03T00:00:00+02:00",
    "duration": "PT8H"
  }
]
'''

# Query and save to calendar
The saver agent receives the draft id and queries the database for that draft and then saves it to the calendar.

In [ ]:
from datetime import date as Date
from sqlmodel import SQLModel, create_engine, Session, select
import json

# Clear SQLModel metadata
SQLModel.metadata.clear()
from fateforger.agents.schedular.models.new_models import CalendarEvent, ScheduleDraft, DraftStore
events = [CalendarEvent.model_validate(event) for event in json.loads(timebox_json)]
from datetime import datetime
today = datetime.now()
# Now import our models
# from fateforger.agents.schedular.models.calendar_event import CalendarEvent
# from fateforger.agents.schedular.models.schedule_draft import ScheduleDraft, DraftStore

from pydantic import TypeAdapter
from typing import List

draft  = ScheduleDraft(date=today, events=events)
draft.finalise()

# Create engine and tables
engine = create_engine("sqlite:///:memory:", echo=False)
SQLModel.metadata.create_all(engine)
with Session(engine) as session:
    store = DraftStore(session)
    saved_draft = store.save(draft) # TODO: make sure i get the id for querying later

In [5]:
# now query the draft
with Session(engine) as session:
    store = DraftStore(session)
    queried_draft = store.get_by_id(saved_draft.id)

AttributeError: 'DraftStore' object has no attribute 'get'

In [3]:
import asyncio
import logging
from typing import Any, List

from tenacity import (
    retry,
    retry_any,
    retry_if_exception_type,
    retry_if_result,
    stop_after_attempt,
    wait_exponential,
    after_log,
)

class EventSubmitter:
    """
    Async batch submitter for MCP events, Python 3.10 edition:
      - per-call timeout via asyncio.wait_for()
      - exponential-backoff retries on exceptions or result.is_error
      - bounded parallelism (Semaphore)
      - gathers all tasks concurrently (asyncio.gather)
    """

    def __init__(
        self,
        mcp: Any,
        *,
        max_concurrency: int = 5,
        timeout: float = 10.0,
        retries: int = 5,
        base_delay: float = 0.5,
        max_delay: float = 5.0,
    ):
        self._mcp = mcp
        self._sem = asyncio.Semaphore(max_concurrency)
        self._timeout = timeout
        self._logger = logging.getLogger(self.__class__.__name__)
        self._retries = retries
        self._base_delay = base_delay
        self._max_delay = max_delay

        # retry on exception OR when .is_error is True
        retry_rule = retry_any(
            retry_if_exception_type(Exception),
            retry_if_result(lambda r: getattr(r, "is_error", False)),
        )

        async def _raw_rpc(event):
            # enforce per-call timeout in 3.10
            return await asyncio.wait_for(
                self._mcp.call_tool(
                    "create-event",
                    arguments=event.model_dump(exclude_none=True),
                ),
                timeout=self._timeout,
            )

        # wrap it in Tenacity’s retry decorator
        self._rpc = retry(
            retry=retry_rule,
            stop=stop_after_attempt(self._retries),
            wait=wait_exponential(multiplier=self._base_delay, max=self._max_delay),
            after=after_log(self._logger, logging.WARNING),
            reraise=True,
        )(_raw_rpc)

    async def _invoke_with_throttle(self, event) -> Any | None:
        """Acquire semaphore, call RPC (with retry+timeout), catch final failures."""
        async with self._sem:
            try:
                return await self._rpc(event)
            except Exception as exc:
                ev_id = getattr(event, "id", "<unknown>")
                self._logger.error("Event %s failed permanently: %s", ev_id, exc)
                return None  # keep slot in result list

    async def submit(self, draft) -> List[Any | None]:
        """
        Launch all create-event calls in parallel (up to max_concurrency).
        Returns a list of responses or None for each permanently failed event.
        """
        # fire off all workers
        tasks = [
            asyncio.create_task(self._invoke_with_throttle(ev))
            for ev in draft.events
        ]
        # wait for them all to finish, preserving order
        return await asyncio.gather(*tasks)


In [7]:
# from autogen_ext.tools.mcp import MCPWorkbench, StreamableHttpServerParams
from fateforger.tools.calendar_mcp import get_calendar_mcp_tools
import os
SERVER_URL = os.getenv("CALENDAR_MCP_URL", "http://localhost:3000")

from autogen_ext.tools.mcp import McpWorkbench
from autogen_ext.tools.mcp import StreamableHttpServerParams, mcp_server_tools



mcp_workbench = McpWorkbench(StreamableHttpServerParams(
        url=SERVER_URL,
        timeout=5,
    ))

In [6]:
from datetime import datetime, timedelta
from pydantic import BaseModel


class DayStartEnd(BaseModel):
    timeMin: str
    timeMax: str


def get_day_start_end(delta:int=0) -> DayStartEnd:
    """
    Get the start and end of the day in ISO format.
    Replace with the desired date if needed.
    """
    # Use today's date or replace with a specific date
    today = datetime.today()
    day = today + timedelta(days=delta)

    # early morning, ie 00:00:00
    day_start = day.replace(hour=0, minute=0, second=0, microsecond=0,tzinfo=None).isoformat()
    day_end = day.replace(hour=23, minute=59, second=59, microsecond=0,tzinfo=None).isoformat()

    return DayStartEnd(
        timeMin=day_start,
        timeMax=day_end
    )
get_day_start_end().timeMin

'2025-08-03T00:00:00'

In [1]:
from fateforger.agents.schedular.models.new_models import EventType


EventType.get_event_type_from_color_id("9")

<EventType.DEEP_WORK: 'DW'>

In [8]:
import re
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, Field
from dataclasses import dataclass

class ParsedCalendarEvent(BaseModel):
    """Structured representation of a calendar event from MCP output."""
    title: str
    event_id: str
    start: datetime
    end: datetime
    view_url: str
    timezone: str = "GMT+2"

class CalendarEventParser:
    """Deterministic parser for Google Calendar MCP text output."""
    
    # Regex patterns for parsing the structured text
    EVENT_PATTERN = re.compile(
        r'(\d+)\.\s+Event:\s+(.+?)\n'
        r'Event ID:\s+([^\n]+)\n'
        r'Start:\s+(.+?)\n'
        r'End:\s+(.+?)\n'
        r'View:\s+([^\n]+)',
        re.MULTILINE | re.DOTALL
    )
    
    DATETIME_PATTERN = re.compile(
        r'(\w{3}),\s+(\w{3})\s+(\d{1,2}),\s+(\d{4}),\s+(\d{1,2}):(\d{2})\s+(AM|PM)\s+(GMT[+-]\d+)'
    )
    
    MONTH_MAP = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    @classmethod
    def parse_datetime(cls, datetime_str: str) -> datetime:
        """Parse 'Sun, Aug 3, 2025, 1:30 AM GMT+2' format to datetime."""
        match = cls.DATETIME_PATTERN.match(datetime_str.strip())
        if not match:
            raise ValueError(f"Cannot parse datetime: {datetime_str}")
        
        day_name, month_name, day, year, hour, minute, ampm, timezone = match.groups()
        
        # Convert to 24-hour format
        hour = int(hour)
        if ampm == 'PM' and hour != 12:
            hour += 12
        elif ampm == 'AM' and hour == 12:
            hour = 0
            
        return datetime(
            year=int(year),
            month=cls.MONTH_MAP[month_name],
            day=int(day),
            hour=hour,
            minute=int(minute)
        )
    
    @classmethod
    def parse_events_text(cls, text: str) -> List[ParsedCalendarEvent]:
        """Parse the complete calendar events text output."""
        events = []
        
        # Extract the main content after "Found X event(s):"
        content_start = text.find(':\n\n') + 3
        if content_start == 2:  # Not found
            content_start = 0
            
        content = text[content_start:]
        
        # Find all event matches
        matches = cls.EVENT_PATTERN.findall(content)
        
        for match in matches:
            event_num, title, event_id, start_str, end_str, view_url = match
            
            try:
                start_dt = cls.parse_datetime(start_str)
                end_dt = cls.parse_datetime(end_str)
                
                event = ParsedCalendarEvent(
                    title=title.strip(),
                    event_id=event_id.strip(),
                    start=start_dt,
                    end=end_dt,
                    view_url=view_url.strip()
                )
                events.append(event)
                
            except Exception as e:
                print(f"Warning: Failed to parse event {event_num}: {e}")
                continue
                
        return events



In [ ]:
# Initialize the MCP workbench with the tools
from typing import List
from fateforger.agents.schedular.models.new_models import CalendarEvent

async def get_events_from_calendar(day_delta: int = 0) -> List[CalendarEvent]:
    """
    Get events from the calendar for the current day.
    """
    day_start_end = get_day_start_end(day_delta)

    tool_result = await mcp_workbench.call_tool(
        "list-events",
        arguments=dict(
            calendarId="primary",
            timeMin=day_start_end.timeMin,
            timeMax=day_start_end.timeMax
        )
    )
    if tool_result.is_error:
        raise RuntimeError(f"Failed to get events: {tool_result.error_message}")

    return CalendarEventParser.parse_events_text(tool_result.result[0].content)
result = await get_events_from_calendar()

result

In [17]:
result

[ParsedCalendarEvent(title='Sleep', event_id='r1o8ukp5drrj3p70ofe7kvhcgs', start=datetime.datetime(2025, 8, 3, 1, 30), end=datetime.datetime(2025, 8, 3, 9, 30), view_url='https://www.google.com/calendar/event?eid=cjFvOHVrcDVkcnJqM3A3MG9mZTdrdmhjZ3MgaHVnby5ldmVyc0Bt', timezone='GMT+2'),
 ParsedCalendarEvent(title='morning ritual', event_id='5pdn8qfld6ctcvqoj7585nen9q', start=datetime.datetime(2025, 8, 3, 9, 45), end=datetime.datetime(2025, 8, 3, 10, 45), view_url='https://www.google.com/calendar/event?eid=NXBkbjhxZmxkNmN0Y3Zxb2o3NTg1bmVuOXEgaHVnby5ldmVyc0Bt', timezone='GMT+2'),
 ParsedCalendarEvent(title='deep work on c2t', event_id='6vbkp2g69eqm03sscejqd82nn7', start=datetime.datetime(2025, 8, 3, 10, 45), end=datetime.datetime(2025, 8, 3, 13, 0), view_url='https://www.google.com/calendar/event?eid=NnZia3AyZzY5ZXFtMDNzc2NlanFkODJubjcgaHVnby5ldmVyc0Bt', timezone='GMT+2'),
 ParsedCalendarEvent(title='Eten bij ouders', event_id='5pm3ernm4hpcqrpc30ni4bc4rg', start=datetime.datetime(2025, 8,

In [ ]:
await mcp_workbench.call_tool(
        "list-events",
        arguments=dict(
            calendarId="primary",
            timeMin=day_start_end.timeMin,
            timeMax=day_start_end.timeMax
        )

In [13]:
from typing import List
from fateforger.agents.schedular.models.new_models import CalendarEvent

async def search_events_from_calendar(query:str, day_delta: int = 0) -> List[CalendarEvent]:
    """
    Get events from the calendar for the current day.
    """
    day_start_end = get_day_start_end(day_delta)

    tool_result = await mcp_workbench.call_tool(
        "search-events",
        arguments=dict(
            query=query,
            calendarId="primary",
            timeMin=day_start_end.timeMin,
            timeMax=day_start_end.timeMax
        )
    )
    if tool_result.is_error:
        raise RuntimeError(f"Failed to get events: {tool_result.error_message}")

    return tool_result

In [22]:
await search_events_from_calendar("c2t", day_delta=0)

ToolResult(type='ToolResult', name='search-events', result=[TextResultContent(type='TextResultContent', content='Found 1 event(s) matching your search:\n\n1. Event: deep work on c2t\nEvent ID: 6vbkp2g69eqm03sscejqd82nn7\nStart: Sun, Aug 3, 2025, 10:45 AM GMT+2\nEnd: Sun, Aug 3, 2025, 1:00 PM GMT+2\nView: https://www.google.com/calendar/event?eid=NnZia3AyZzY5ZXFtMDNzc2NlanFkODJubjcgaHVnby5ldmVyc0Bt')], is_error=False)

In [8]:
await mcp_workbench.list_tools()

[{'name': 'list-calendars',
  'description': 'List all available calendars',
  'parameters': {'type': 'object',
   'properties': {},
   'required': [],
   'additionalProperties': False}},
 {'name': 'list-events',
  'description': 'List events from one or more calendars.',
  'parameters': {'type': 'object',
   'properties': {'calendarId': {'type': 'string',
     'description': 'ID of the calendar(s) to list events from. Accepts either a single calendar ID string or an array of calendar IDs (passed as JSON string like \'["cal1", "cal2"]\')'},
    'timeMin': {'type': 'string',
     'description': "Start time boundary. Preferred: '2024-01-01T00:00:00' (uses timeZone parameter or calendar timezone). Also accepts: '2024-01-01T00:00:00Z' or '2024-01-01T00:00:00-08:00'."},
    'timeMax': {'type': 'string',
     'description': "End time boundary. Preferred: '2024-01-01T23:59:59' (uses timeZone parameter or calendar timezone). Also accepts: '2024-01-01T23:59:59Z' or '2024-01-01T23:59:59-08:00'."

In [5]:
# configure logging if you want to see warnings/errors
import logging
logging.basicConfig(level=logging.INFO)

submitter = EventSubmitter(
    mcp_workbench,
    max_concurrency=5,
    timeout=10.0,
    retries=5,
    base_delay=0.5,
    max_delay=5.0,
)

results = await submitter.submit(draft)
# results[i] is either the successful response or None if it failed after all retries


INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-03-26
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-03-26
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 202 Accepted"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:3000 "HTTP/1.1 200 OK"
INFO